In [1]:
# Assignment_2: Akhilesh Ravi-16110007
#Using Jane Austen Novels for Language Model Generation

# Importing all packages
import math
import numpy as np
from operator import itemgetter
from nltk import word_tokenize, sent_tokenize
from string import punctuation
from nltk.util import ngrams

In [2]:
import re
from sklearn.model_selection import train_test_split 

# Pre-Processing of the dataset

DATASET- Jane Austen Novels: The Complete Works of Jane Austen take one line, neglect empty line and then add it to line_set then lower down whole set(string)


In [3]:
with open("JaneAusten.txt", 'r') as file:  
    lines = ['']
    for line in (l.rstrip() for l in file):
        if line != '' or lines[-1] != '\n':                 
            lines.append(line + '\n')
    text = "".join(lines)
text = text.lower()  

# Tokenizing sentences
# Vocabulary dictionary and corpus
vocabulary = {}
corpus = []
list_sentence = sent_tokenize(text)
# <s> is added to the start of each sentence
# </s> is added to the end of sentences
for sentence in list_sentence:
    word_list = word_tokenize(sentence) # tokenizing each word of that sentence
    final_sentence = ['<s>']
    for word in word_list:
        # Obtains the frequency of the words in the corpus
        if len(set(word).intersection(punctuation)) == 0 and (len(word)>1 or word == 'a' or word == 'i') :
            final_sentence.append(word)
            if word not in vocabulary.keys(): 
                vocabulary[word] = 1
            else:
                vocabulary[word] += 1
                
    final_sentence += ['</s>']
    corpus += [" ".join(final_sentence)] #adding processed sentence to the corpus

#Now Processed Corpus is ready    
number_of_sentence = len(corpus)
# Vocabulary list doesn't contain the START and END word used in Pre-Processing
vocabulary['<s>'] = 2*number_of_sentence
vocabulary['</s>'] = 2*number_of_sentence
# Now we have Processed sentence corpus and vocabulary list. 
#Processed Corpus DATA ANALYSIS  
tokens = sum(vocabulary.values())
types = len(vocabulary)
print ('Number of Sentences = ' + str(number_of_sentence))
print ('Number of Types = ' + str(types))
print ('Number of Tokens = ' + str(tokens))
print ("TTR= " + str(types/tokens))
# train-test split in 80:20 ratio
train_data, test_data = train_test_split(corpus, test_size=0.20, random_state=64)   
    

Number of Sentences = 35056
Number of Types = 14651
Number of Tokens = 914639
TTR= 0.01601834166266691


In [4]:
corpus[:12]

['<s> project gutenberg the complete works of jane austen by jane austen this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever </s>',
 '<s> you may copy it give it away or it under the terms of the project gutenberg license included with this ebook or online at title the complete project gutenberg works of jane austen author jane austen editor david widger release date january 25 2010 ebook 31100 language english character set encoding ascii start of this project gutenberg ebook the works of jane austen produced by many project gutenberg volunteers </s>',
 '<s> the works of jane austen edited by david widger project gutenberg editions dedication this jane austen collection is dedicated to alice goodson hart woodby note the accompanying html file has active links to all the volumes and chapters in this set </s>',
 '<s> contents persuasion northanger abbey mansfield park emma lady susan love and freindship and other early works pride and preju

# Q1a MLE for N-Grams : N=  1,2,3,4

In [5]:
# Unigrams
probability_unigram = {}           
for i in vocabulary.keys():
    probability_unigram[i] = (float(vocabulary[i]))/tokens     


# Bigrams

probability_bigram = {} # bigram probabilities
bigram_list = {} # biagram list
# from each sentence in corpus extracting all the biagrams
for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(1,len(words_in_sentence)):
        j = (words_in_sentence[i-1], words_in_sentence[i])#taking all consecutive combination
        try:
            bigram_list[j] += 1
        except:
            bigram_list[j] = 1
             
for i in bigram_list.keys():
    probability_bigram[i] = float(bigram_list[i])/vocabulary[i[0]]


# Trigrams
probability_trigram = {} #trigram probabilities
trigram_list = {}

for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(2,len(words_in_sentence)):
        j = (words_in_sentence[i-2], words_in_sentence[i-1], words_in_sentence[i])
        try:
            trigram_list[j] += 1
        except:
            trigram_list[j] = 1


for i in trigram_list.keys():
    probability_trigram[i] = (trigram_list[i])/bigram_list[(i[0], i[1])]

# Quadrigrams
probability_quadgram = {}
quadgram_list = {}

for sentence in corpus:
    words_in_sentence = sentence.split()
    for i in range(3,len(words_in_sentence)):
        j = (words_in_sentence[i-3], words_in_sentence[i-2], words_in_sentence[i-1], words_in_sentence[i])
        try:
            quadgram_list[j] += 1
        except:
            quadgram_list[j] = 1

#NOw we have list a histogram of QUADgram in the corpus             
for i in quadgram_list.keys():
    probability_quadgram[i] = (quadgram_list[i]*1.0)/trigram_list[(i[0], i[1], i[2])]

In [6]:
print("UNIGRAM-MLE - Top 15\n")
for i in sorted(probability_unigram.items(), key=lambda x:x[1], reverse=True)[:15]:
    print(i)
    
print("\nBIGRAM-MLE - Top 15\n")
for i in sorted(probability_bigram.items(), key=lambda x:x[1], reverse=True)[:15]:
    print(i)
    
print("\nTRIGRAM-MLE - Top 15\n")
for i in sorted(probability_trigram.items(), key=lambda x:x[1], reverse=True)[:15]:
    print(i)

print("\nQUADGRAM-MLE - Top 15\n")
for i in sorted(probability_quadgram.items(), key=lambda x:x[1], reverse=True)[:15]:
    print(i)

UNIGRAM-MLE - Top 15

('<s>', 0.07665537988211743)
('</s>', 0.07665537988211743)
('the', 0.03128994062138177)
('to', 0.028454942332439354)
('and', 0.02644321967464759)
('of', 0.025319278972359586)
('a', 0.015726423211780822)
('her', 0.015204905979298937)
('i', 0.015052933452433146)
('in', 0.013320009315150568)
('was', 0.012856438441833335)
('it', 0.0118308972173721)
('she', 0.011605671745901935)
('not', 0.010653383466045073)
('that', 0.009639868844429331)

BIGRAM-MLE - Top 15

(('whatsoever', '</s>'), 1.0)
(('david', 'widger'), 1.0)
(('2010', 'ebook'), 1.0)
(('31100', 'language'), 1.0)
(('encoding', 'ascii'), 1.0)
(('edited', 'by'), 1.0)
(('dedication', 'this'), 1.0)
(('goodson', 'hart'), 1.0)
(('woodby', 'note'), 1.0)
(('html', 'file'), 1.0)
(('patents', 'there'), 1.0)
(('1760', 'married'), 1.0)
(('1784', 'elizabeth'), 1.0)
(('stevenson', 'esq'), 1.0)
(('esq', '</s>'), 1.0)

TRIGRAM-MLE - Top 15

(('gutenberg', 'the', 'complete'), 1.0)
(('complete', 'works', 'of'), 1.0)
(('austen', 'b

Q1a

In [7]:

def NGrams(N):
    possible = types**N
    NGram_list = []
    for sentence in corpus:
        lst = list(ngrams(sentence.split(), N))
        NGram_list += lst
    unique_NGrams = set(NGram_list)
    present = len(unique_NGrams)
    return possible, present

for i in range(1,5):
    print('N:',i)
    possible, present = NGrams(i)
    print ("Number of possible n-grams: " + str(possible))
    print ("Number of present n-grams: " + str(present))
    print()

N: 1
Number of possible n-grams: 14651
Number of present n-grams: 14651

N: 2
Number of possible n-grams: 214651801
Number of present n-grams: 213027

N: 3
Number of possible n-grams: 3144863536451
Number of present n-grams: 527634

N: 4
Number of possible n-grams: 46075395672543601
Number of present n-grams: 672052



Q1b

In [8]:
#Generator(type_of_model) for sentence generation 

# Sampling Function from the Multinomial is used

def Generator(model_ngram):
    sentence = "<s>"
    start = "<s>"
    if model_ngram=="Unigram":
        word_probability = list(probability_unigram.values())
        word_list = list(probability_unigram.keys())
        while(True):
            lst = np.random.multinomial(1,word_probability)
            word = word_list[int(np.where(lst == 1)[0])]
            if (word=="<s>"):
                continue
            elif (word == "</s>"):
                sentence += word
                print (sentence)
                break
            else:
                sentence += word + " "
                
    elif model_ngram=="Bigram":
        while(True):
            probabilities, next_word = [], []
            for i in probability_bigram.keys():
                if(i[0]==start):
                    probabilities.append(probability_bigram[i])
                    next_word.append(i[1])
                    # ValueError: sum(pvals[:-1]) > 1.0 may occur
                    # This is because the precision of the numbers in Python are not exact
                    # Please Change your system to 32-bit and run it again. Or run this part by part
            lst = (np.random.multinomial(1.0,probabilities))
            word = next_word[int(np.where(lst == 1)[0])]
            if word == "</s>":
                sentence += word
                print (sentence)
                break
            else:
                start = word
                sentence += word + " "
                
    elif model_ngram=="Trigram":
        probabilities, next_word = [], []
        for i in probability_bigram.keys():
            if(i[0]==start):
                probabilities.append(probability_bigram[i])
                next_word.append(i[1])                
            
        lst = np.random.multinomial(1,probabilities)
        word = next_word[int(np.where(lst==1)[0])]
        bigTuple = (start,word)
        sentence += word + " "
            
        while(True):
            next_wordT, probabilitiesT = [], []
            for i in probability_trigram.keys():
                if(i[0]==bigTuple[0] and i[1] == bigTuple[1]):
                    probabilitiesT.append(probability_trigram[i])
                    next_wordT.append(i[2])
                    
            
            lstT = np.random.multinomial(1,probabilitiesT)
            wordT = next_wordT[int(np.where(lstT==1)[0])]
            bigTuple = (bigTuple[1],wordT)
            
            if wordT == "</s>":
                sentence += wordT
                print (sentence)
                break
            else:
                sentence += wordT + " "
                
                
    elif model_ngram == "Quadgram":
        next_wordB, probabilitiesB = [], []
        for i in probability_bigram.keys():
            if(i[0]==start):
                probabilitiesB.append(probability_bigram[i])
                next_wordB.append(i[1])                
            
        lstB = np.random.multinomial(1,probabilitiesB)
        wordB = next_wordB[int(np.where(lstB==1)[0])]
        bigTuple = (start,wordB)
        sentence += wordB + " "
            
        next_wordT, probabilitiesT =[], []
        for i in probability_trigram.keys():
            if(i[0] == bigTuple[0] and i[1]== bigTuple[1]):
                next_wordT.append(i[2])
                probabilitiesT.append(probability_trigram[i])
        
        lstT = np.random.multinomial(1,probabilitiesT)
        wordT = next_wordT[int(np.where(lstT==1)[0])]
        trigTuple = (start,wordB,wordT)
        sentence += wordT + " "
            
        while(True):
            next_wordQ, probabilitiesQ = [], []
            for i in probability_quadgram.keys():
                if(i[0]==trigTuple[0] and i[1] == trigTuple[1] and i[2]==trigTuple[2]):
                    probabilitiesQ.append(probability_quadgram[i])
                    next_wordQ.append(i[3])
                    
            
            lstQ = np.random.multinomial(1,probabilitiesQ)
            wordQ = next_wordQ[int(np.where(lstQ==1)[0])]
            trigTuple = (trigTuple[1],trigTuple[2],wordQ)       


            if wordQ == "</s>":
                sentence += wordQ
                print (sentence)
                break
            else:
                sentence += wordQ + " "

Q1c - Evaluation

In [10]:
# PERPLEXITY
# Code For Calculating PERPLEXITY
# Bigram model
# Uses Add-one smoothing

bigram_List_Add1, bigram_Add1 = {}, {}
change = {} # delta change in the count
for i in bigram_list:
    bigram_Add1[i] = float(bigram_list[i] + 1)/(vocabulary[i[0]] + types)
    bigram_List_Add1[i] = bigram_Add1[i]*vocabulary[i[0]]
    change[i] = bigram_list[i] - bigram_List_Add1[i]

    
def Bigrams_Test():
    ngrams_list = []
    for s in test_data:
        ngrams_list += list(ngrams(s.split(" "), 2))
    return ngrams_list
bigrams_Test = Bigrams_Test()

test_size = 0
for sentence in test_data:
    test_size += len(sentence.split(" "))

perplexity_Add1 = 1.0
for k in bigrams_Test:
    if(k not in bigram_Add1):
        perplexity_Add1 *= (len(unigrams) ** (1.0/test_size))
    else:
        perplexity_Add1 *= ((1.0/bigram_Add1[k]) ** (1.0/test_size))
    

print ("Perplexity for Add-1 smoothing NGram Language Model=", str(perplexity_Add1))

Perplexity for Add-1 smoothing NGram Language Model= 656.2864111198207


Generating 5 sentences

In [11]:


print (" UNIGRAM SENTENCES \n")
for z in range(5):
    Generator("Unigram")  
print ("\n Readibility is very bad for UNIGRAMS, the sentences are completely random \n\n")
    
    
print (" BIGRAM SENTENCES\n")
for z in range(5):
    Generator("Bigram")
print ("\n Readibility is poor for BIGRAMS (some context is there) \n\n")

print (" TRIGRAM SENTENCES\n")
for z in range(5):
    Generator("Trigram")
print ("\n Readibility is better for TRIGRAMS \n\n")

print (" QUADGRAM SENTENCES\n")
for z in range(5):
    Generator("Quadgram")
print ("\n Readibility is fine for QUADGRAMS \n")
    

 UNIGRAM SENTENCES 

<s>not the may such not for nobody that all </s>
<s></s>
<s>she know and at of be have fine certainly struck when i rosings as their angry whose in about of whose your him flow were that great but i see by so and there question her than be gate </s>
<s>been her ready </s>
<s>observing </s>

 Readibility is very bad for UNIGRAMS, the sentences are completely random 


 BIGRAM SENTENCES

<s>professor michael hart </s>
<s>well as she could make me out of communication in want to meet them bingley called on a little while they were sent off as she saw him marry and no disservice with all things but when frank churchill had returned home was not have been with them and given for me that it so full of something more than i would convey she was up these in conduit street especially as far behind the winter and expense of having an engagement to assure you had done </s>
<s>well at her more to get all and i used to cole and glad of sisterly partiality provoked her commendat